In [1]:
from torch import cuda

device = 'cuda' if cuda.is_available() else 'cpu'

In [2]:
!git clone https://github.com/xxbean/pozalabs

Cloning into 'pozalabs'...
remote: Enumerating objects: 91, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 91 (delta 1), reused 0 (delta 0), pack-reused 69
Unpacking objects: 100% (91/91), 64.63 MiB | 5.43 MiB/s, done.


In [4]:
import sys
sys.path.append("/content/pozalabs/ComMU-code")
sys.path.append("/content/pozalabs/Multi-Scale-1D-ResNet")
from model.multi_scale_ori import *
from model.meta_labeling import * #여기에 메타 데이터 함수 정보 있음
import torch
import numpy as np
from torchsummary import summary
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import tqdm
import torch.nn.functional as F
import sys
import glob

In [9]:

%cd /content/pozalabs/
%ls

/content/pozalabs
ComMU-code/  Multi-Scale-1D-ResNet/  README.md


In [10]:
import glob


raw_train = glob.glob("./ComMU-code/dataset/commu_midi/train/raw/**")
raw_val = glob.glob("./ComMU-code/dataset/commu_midi/val/raw/**")
train_meta_npy_ = np.load("./ComMU-code/dataset/commu_midi/output_npy_/input_train.npy", allow_pickle=True)
train_midi_npy_ = np.load("./ComMU-code/dataset/commu_midi/output_npy_/target_train.npy", allow_pickle=True)
val_meta_npy_ = np.load("./ComMU-code/dataset/commu_midi/output_npy_/input_val.npy", allow_pickle=True)
val_midi_npy_ = np.load("./ComMU-code/dataset/commu_midi/output_npy_/target_val.npy", allow_pickle=True)

label_list = np.unique(train_meta_npy_[:,0])

In [10]:
# !unzip /content/drive/MyDrive/ComMU-code/dataset/commu_midi/output_npy.zip -d /content/drive/MyDrive/ComMU-code/dataset/commu_midi

Archive:  /content/drive/MyDrive/ComMU-code/dataset/commu_midi/output_npy.zip
replace /content/drive/MyDrive/ComMU-code/dataset/commu_midi/output_npy_/input_train.npy? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: /content/drive/MyDrive/ComMU-code/dataset/commu_midi/output_npy_/input_train.npy  
replace /content/drive/MyDrive/ComMU-code/dataset/commu_midi/output_npy_/target_val.npy? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [12]:
class Commu(Dataset):
    def __init__(self, meta_npy, midi_npy,seq_len):
        self.meta_npy = meta_npy
        self.midi_npy = midi_npy
        self.seq_len = seq_len
        self.label_npy = np.zeros_like(self.meta_npy)
        for i in range(11):
            self.label_npy[:,i] = np.array(list(map(meta_list[i],meta_npy[:,i])))

    def __len__(self):
        return len(self.meta_npy)

    def __getitem__(self, idx):
        label = self.label_npy[idx]
        midi = np.zeros((1,self.seq_len))
        midi_real = self.midi_npy[idx][:self.seq_len]
        midi[:,:len(midi_real)] = midi_real
        midi = torch.tensor(midi.tolist(),dtype=torch.float)
        label = torch.tensor(label.tolist(),dtype=torch.float)
        return midi,label

In [13]:
real_data = Commu(train_meta_npy_, train_midi_npy_,512)
real_loader = DataLoader(real_data, batch_size=256, shuffle=True)
val_data = Commu(val_meta_npy_, val_midi_npy_,512)
val_loader = DataLoader(val_data, batch_size=256, shuffle=True)

In [14]:
test_batch = next(iter(real_loader))[0]
test_target = next(iter(real_loader))[1]

In [15]:
msresnet = MSResNet(input_channel=1, layers=[1, 1, 1, 1], num_classes=5)
msresnet = msresnet.cuda()
# summary(msresnet,(1,512))

In [16]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(msresnet.parameters(), lr=0.001, momentum=0.9)
num_epochs = 100
meta_num = 1

In [17]:
pred = msresnet(test_batch.cuda())[0].argmax(1).cpu()
print(pred)
print(test_target[:,0])
sum(test_target[:,0]==pred)

tensor([1, 2, 1, 1, 0, 4, 1, 1, 0, 0, 0, 0, 2, 1, 0, 1, 1, 2, 1, 1, 4, 4, 1, 0,
        4, 0, 1, 0, 2, 1, 1, 1, 4, 0, 1, 2, 0, 2, 1, 1, 0, 0, 1, 2, 4, 3, 2, 1,
        0, 1, 0, 1, 3, 0, 1, 1, 4, 0, 0, 2, 2, 0, 2, 1, 0, 0, 4, 0, 0, 0, 0, 2,
        0, 0, 0, 2, 2, 4, 0, 1, 0, 2, 0, 1, 4, 4, 0, 2, 0, 0, 0, 3, 1, 0, 0, 2,
        0, 0, 2, 1, 0, 0, 1, 2, 2, 0, 0, 1, 2, 2, 4, 0, 4, 0, 1, 0, 2, 4, 2, 1,
        0, 1, 0, 1, 0, 0, 2, 1, 0, 0, 1, 1, 0, 0, 4, 0, 2, 1, 4, 1, 0, 0, 2, 2,
        0, 4, 2, 4, 0, 1, 2, 0, 0, 1, 1, 1, 1, 0, 2, 4, 4, 0, 4, 1, 4, 0, 0, 1,
        2, 1, 4, 4, 4, 0, 0, 1, 4, 2, 0, 4, 1, 2, 2, 1, 2, 1, 1, 4, 2, 2, 2, 1,
        0, 1, 0, 0, 2, 4, 1, 2, 2, 0, 0, 0, 0, 0, 0, 4, 2, 0, 1, 4, 0, 0, 4, 2,
        2, 4, 0, 1, 4, 4, 0, 0, 1, 0, 4, 4, 0, 0, 0, 1, 0, 0, 1, 1, 4, 0, 0, 1,
        0, 4, 0, 1, 2, 0, 2, 4, 1, 0, 0, 0, 1, 1, 0, 1])
tensor([4., 6., 7., 3., 4., 2., 7., 3., 6., 6., 7., 3., 7., 2., 6., 3., 2., 4.,
        3., 5., 7., 3., 3., 4., 2., 6., 4., 3., 4., 5., 6., 5.,

tensor(17)

In [18]:
for epoch in range(num_epochs):  

    running_loss = 0.0
    for i, data in enumerate(tqdm.tqdm(real_loader)):
        msresnet.train()
        # get the inputs; data is a list of [inputs, labels]
        midi,label = data
        midi = midi.cuda()
        label = label.cuda()[:,meta_num]
#         label = F.one_hot(label.to(torch.int64), num_classes = 10)
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = msresnet(midi)[0]
        loss = criterion(outputs, label.long())
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0
    print(running_loss / 41)
    if epoch %6 == 5:
        val_num = 0
        correct_num = 0
        with torch.no_grad():
            msresnet.eval()
            for i, data in enumerate(val_loader):
                midi,label = data
                midi = midi.cuda()
                label = label[:,meta_num]
                num = label.shape[0]
                val_num += num
                outputs = msresnet(midi)[0].argmax(1).cpu()
                correct_num += sum(outputs==label)
                
            print(f"validation accuracy for epoch {epoch} is : {correct_num/val_num}")
        

print('Finished Training')

100%|██████████| 41/41 [00:03<00:00, 13.36it/s]


0.7445349358930821


 85%|████████▌ | 35/41 [00:01<00:00, 17.74it/s]


KeyboardInterrupt: ignored